# Hermes spurious event rate

The question:

_Given a source flux F, what's the probability of observing at least two photons over separate, coupled SDD channel, within a coincidence window $\Delta t$?_ 


## Montecarlo approach

The flux of a bright burst is $F' = 100$ photons/cm$^2$/s.
Hence, integrating over the HERMES geometric area, the flux over a couple of SDD is ($\approx 0.9$ cm$^2$) is $F = 90 $ photons/s.

In [34]:
from scipy.stats import expon
import numpy as np

F = 90 # photons/s, flux over a couple of SDD

N = 1_000_000 # simulation size
delays = expon(scale=1 / F).rvs(N) # the wait times
on_same_sdd = np.random.choice([True, False], size=N) # True if the photons fell on the same SDD as the one preceding it.

coincidence_window = 15 * 10 **-6

f = len(delays[(delays < coincidence_window) & (on_same_sdd == False)]) / N
print(f"Fraction of spurios event is : {f:.5f}")

Fraction of spurios event is : 0.00067


## Math

The probability of having a spurious event can be expressed as the conditional probability $P_A$ of counting one or more event over a coincidence window at cell 1; given that we did count one event over a small interval immediately before, at cell 2. Since the two events are independent, we have:

$$P = P(A | B) = \frac{P(A \wedge B)}{(B)} = \frac{P(A) P(B)}{P(B)} = P(A)$$

Hence, noting $P_\lambda(k)$ the Poisson probability of observing one count over one coincidence window:

$$P = P(A) = 1 - P_{\lambda}(k = 0) = 1 - \lambda^0 \frac{e^{-\lambda}}{0!} = 1 - e^{-\lambda}$$

where the average count over the window lenght is $\lambda = \frac{F \cdot \Delta t}{2}$, and:

$$ P = 1 - e^{- \frac{F \cdot \Delta t}{2}} $$

In [49]:
print(f"theoretical probability is {1 - np.exp(- F * coincidence_window / 2):.5f}.")

theoretical probability is 0.00067.


Alternative approach. The probability $P$ of observing two counts over the scintillator is equal to the probability of observing the two count over distinct channels plus the probability of observing the two count over different channels (the two events are mutually exclusive).

$$ P = P_{\text{same}} + P_{\text{distinct}}$$

By symmetry:

$$P_{\text{same}} = P_{\text{distinct}}$$

So:

$$  P_{\text{distinct}} = \frac{P}{2} = \frac{1}{2} \int_0^{\Delta t} Fe^{-Ft} dt =  \frac{1}{2} (1 - e^{-F\Delta t})$$

In [50]:
print(f"theoretical probability is {0.5 * (1 - np.exp(-F * coincidence_window)):.5f}.")

theoretical probability is 0.00067.


![image info](https://m.media-amazon.com/images/I/71j0uMS2gRL._AC_UF1000,1000_QL80_.jpg)

## TLDR:


I got a count over a SDD one instant before. The probability of having one or more over the next $\Delta t$ is one minus the probability of having none.